# Neo4j와 LangChain을 활용한 영화 추천 시스템

---

## 1. Neo4J AuraDB 환경 설정

In [9]:
import os
from dotenv import load_dotenv

# 환경 변수 로드
load_dotenv()

True

In [10]:
from langchain_neo4j import Neo4jGraph

# LangChain 도구 활용 - DB 연결 객체 초기화 
graph = Neo4jGraph( 
    url=os.getenv("NEO4J_URI"), 
    username=os.getenv("NEO4J_USERNAME"), 
    password=os.getenv("NEO4J_PASSWORD"),
)

In [11]:
# 테스트 쿼리 실행 
cypher_query = """
CREATE (n:Test {name: "Hello AuraDB"}) 
RETURN n
"""

graph.query(cypher_query)

[#2B66]  _: <CONNECTION> error: Failed to read from defunct connection IPv4Address(('si-69205ede-5bd3.production-orch-0033.neo4j.io', 7687)) (ResolvedIPv4Address(('34.78.243.29', 7687))): ConnectionAbortedError(10053, '현재 연결은 사용자의 호스트 시스템의 소프트웨어의 의해 중단되었습니다')
Transaction failed and will be retried in 1.0809969576663634s (Failed to read from defunct connection IPv4Address(('si-69205ede-5bd3.production-orch-0033.neo4j.io', 7687)) (ResolvedIPv4Address(('34.78.243.29', 7687))))
[#2B65]  _: <CONNECTION> error: Failed to read from defunct connection ResolvedIPv4Address(('34.78.243.29', 7687)) (ResolvedIPv4Address(('34.78.243.29', 7687))): ConnectionAbortedError(10053, '현재 연결은 사용자의 호스트 시스템의 소프트웨어의 의해 중단되었습니다')
Unable to retrieve routing information
Transaction failed and will be retried in 2.1479901614599504s (Unable to retrieve routing information)


[{'n': {'name': 'Hello AuraDB'}}]

In [12]:
def reset_database(graph):
    """
    데이터베이스 초기화하기
    """
    # 모든 노드와 관계 삭제
    graph.query("MATCH (n) DETACH DELETE n")
    
    # 모든 제약조건 삭제
    constraints = graph.query("SHOW CONSTRAINTS")
    for constraint in constraints:
        constraint_name = constraint.get("name")
        if constraint_name:
            graph.query(f"DROP CONSTRAINT {constraint_name}")
    
    # 모든 인덱스 삭제
    indexes = graph.query("SHOW INDEXES")
    for index in indexes:
        index_name = index.get("name")
        index_type = index.get("type")
        if index_name and index_type != "CONSTRAINT":
            graph.query(f"DROP INDEX {index_name}")
    
    print("데이터베이스가 초기화되었습니다.")

# 데이터베이스 초기화
reset_database(graph)

데이터베이스가 초기화되었습니다.


---

## 2. 지식 그래프 모델링 및 구축

### 2.1 온톨로지 정의 및 스키마 설계

온톨로지는 특정 도메인 내 개념과 그 관계를 체계적으로 표현하는 지식 구조입니다. Neo4j에서는 제약 조건과 인덱스를 활용하여 데이터 모델을 구현할 수 있습니다.


**노드 타입:**
- Movie: 영화 정보 (id, title, released, rating 속성)
- Person: 감독/배우 정보 (name 속성)
- Genre: 장르 정보 (name 속성)

**관계 타입:**
- DIRECTED: Person(감독) → Movie 관계
- ACTED_IN: Person(배우) → Movie 관계
- IN_GENRE: Movie → Genre 관계

**제약조건/인덱스:**
- Neo4j 데이터베이스에서 **제약조건(Constraint)** 생성으로 데이터 무결성을 보장함
- Movie, Person, Genre 노드의 각 ID와 이름에 **고유성 제약조건**을 설정함
- **인덱스** 생성으로 영화 제목과 개봉일 기반 검색 성능을 최적화함

* 이 스키마는 영화, 인물(감독/배우), 장르 간의 연결 관계를 표현하며, 영화 검색, 추천, 인물 기반 탐색 등 다양한 그래프 기반 쿼리를 지원할 수 있습니다.

---

** 개인적인 견해**
- 이게 온톨로지 혹은 지식그래프 만드는게 별개아니라 그냥 csv 파일 안에 담긴 정보를 가지고 노드/관계 타입을 만드는거뿐임 (수작업으로)


In [13]:
# 제약조건 생성 (데이터 무결성 보장)
# - 각 노드 타입별로 고유성 제약조건을 설정하여 중복 데이터 방지
# - Movie 노드의 id 속성에 고유성 제약조건 설정
# - Person 노드의 name 속성에 고유성 제약조건 설정
# - Genre 노드의 name 속성에 고유성 제약조건 설정
# - IF NOT EXISTS 구문으로 이미 존재하는 경우 오류 방지 
constraints = [
    "CREATE CONSTRAINT movie_id_unique IF NOT EXISTS FOR (m:Movie) REQUIRE m.id IS UNIQUE",
    "CREATE CONSTRAINT person_name_unique IF NOT EXISTS FOR (p:Person) REQUIRE p.name IS UNIQUE",
    "CREATE CONSTRAINT genre_name_unique IF NOT EXISTS FOR (g:Genre) REQUIRE g.name IS UNIQUE"
]

# IF NOT EXISTS: "movie_id_unique" 을 가진 제약 조건이 이미 데이터베이스에 존재하지 않는 경우에만 이 명령을 실행하도록 지정
# FOR (m:Movie): 이 부분은 제약 조건을 적용할 노드의 패턴을 정의 -> Movie 레이블을 가진 노드에 대해 이 제약 조건이 적용됨을 의미
# REQUIRE m.id IS UNIQUE: 이 부분은 제약 조건의 구체적인 요구 사항을 정의 -> Movie 노드의 id 속성이 고유해야 함을 의미

# 인덱스 생성 (검색 성능 최적화)
# - 자주 검색되는 속성에 인덱스를 생성하여 쿼리 성능 향상
# - 영화 제목(title)에 대한 인덱스로 제목 기반 검색 최적화
# - 개봉일(released)에 대한 인덱스로 날짜 기반 검색 및 정렬 최적화
# - IF NOT EXISTS 구문으로 중복 생성 방지

# 인덱스가 필요한 이유: 데이터 조회는 2가지가 존재 -> 풀스캔 vs 인덱스 스캔 - > 인덱싱을하면 칼럼을 따로 정렬된 자료구조(B-Tree, Hash 등)으로 만들어 저장한다.
# 인덱스를 걸어야하는 경우 -> . UNIQUE 제약 속성(유일성 보장 + 탐색 속도 때문에 필수), WHERE에 자주 등장하는 속성, ORDER BY, RANGE 쿼리에 자주 등장하는 속성, JOIN 패턴의 anchor 역할을 하는 속성
# 인덱스를 안 걸어도 되는 경우 -> 거의 검색되지 않는 속성
# 인덱스는 “읽기 속도는 빨라지지만 쓰기 속도는 느려진다” -> B-Tree 같은 자료구조 -> 노드를 추가/수정/삭제할 때마다 인덱스도 다시 정렬 후 업데이트 
# 즉 인덱스를 많이 생성하는 부분은 trade-off 영역에 속한다. 
indexes = [
    "CREATE INDEX movie_title_index IF NOT EXISTS FOR (m:Movie) ON (m.title)",
    "CREATE INDEX movie_release_index IF NOT EXISTS FOR (m:Movie) ON (m.released)",
]

# 제약조건 및 인덱스 실행
# - 정의된 모든 제약조건을 Neo4j 데이터베이스에 적용
for constraint in constraints:  
    graph.query(constraint)

# - 정의된 모든 인덱스를 Neo4j 데이터베이스에 적용
for index in indexes:
    graph.query(index)

print("스키마 설정 완료")

스키마 설정 완료


### 2.2 CSV 데이터로 지식 그래프 구축

- CSV 형태의 영화 데이터셋을 읽어서 그래프 구조로 변환
- 영화, 인물(감독, 배우), 장르를 노드로 변환
- 각 노드 간의 관계(DIRECTED, ACTED_IN, IN_GENRE)를 생성
- 변환된 데이터를 Neo4j 그래프 데이터베이스에 저장

- **CSV 파일 로드**:

    - **TMDB**와 **GroupLens**에서 수집한 데이터를 결합하여 정리
    - 영화 상세정보, 제작진 정보, 키워드는 **TMDB Open API**를 통해 수집됨
    - 영화 링크와 평점 정보는 **공식 GroupLens 웹사이트**에서 획득함
    - 출처: https://www.kaggle.com/code/ibtesama/getting-started-with-a-movie-recommendation-system

In [14]:
# CSV 파일 읽기
import pandas as pd
df = pd.read_csv('data/movies_tmdb_small.csv')
print(df.shape)

df.head(2)

(4803, 10)


,id,released,title,actors,director,genres,rating,overview,runtime,tagline
0,4592,1916-09-04,Intolerance,Lillian Gish|Mae Marsh|Robert Harron|F.A. Turn...,D.W. Griffith,Drama,7.4,"The story of a poor young woman, separated by ...",197.0,The Cruel Hand of Intolerance
1,4661,1925-11-05,The Big Parade,John Gilbert|Renée Adorée|Hobart Bosworth|Clai...,King Vidor,Drama|Romance|War,7.0,The story of an idle rich boy who joins the US...,151.0,NaN


In [ ]:
# 작은 데이터셋으로 학습하려는 경우 아래 코드를 실행 (2010년 이후 영화만 사용)
# df = df[df['released'] >= '2010-01-01']
# print(df.shape)
# df.head(2)

- **필수 라이브러리**:
    - Neo4j 그래프 데이터베이스와 연동하여 문서 처리 및 데이터 분석을 위한 기본 설정

In [15]:
from langchain_neo4j.graphs.graph_document import GraphDocument, Node, Relationship
from langchain_core.documents import Document

- **데이터 변환 (CSV -> 노드/관계)**: 

    1. **Node**: 그래프의 노드(정점)를 표현
    - `id`: 노드의 고유 식별자
    - `type`: 노드의 유형(예: Movie, Person, Genre)
    - `properties`: 노드의 속성들(제목, 이름, 평점 등)

    2. **Relationship**: 두 노드 간의 관계를 표현
    - `source`: 관계의 시작 노드
    - `target`: 관계의 목표 노드
    - `type`: 관계의 유형(예: DIRECTED, ACTED_IN, IN_GENRE)
    - `properties`: 관계의 속성들


In [16]:
"""
밑에 코드가 좀 긴데 대충 설명하자면, 먼저 노드 딕셔너리와 관계 리스트를 초기화하고,
1. CSV 파일의 각 행을 순회하면서 영화, 감독, 배우, 장르에 해당하는 노드를 생성하고,
2. 이들 노드 간의 관계를 설정하는 작업을 수행
위가 전부임
"""


# 중복 노드 생성을 방지하기 위한 딕셔너리 초기화
node_dict = {}  # 노드 ID를 키로 사용하여 생성된 노드 객체를 저장

# 노드 간 관계를 저장할 리스트 초기화
relationships = []

# 배치 처리를 위한 설정
batch_size = 100  # 한 번에 처리할 레코드 수 (메모리가 부족할 경우 더 작은 숫자로 설정)

# 데이터프레임을 배치로 나누어 처리
total_rows = len(df)
for batch_start in range(0, total_rows, batch_size):
    batch_end = min(batch_start + batch_size, total_rows) 
    batch_df = df.iloc[batch_start:batch_end] # 0~100, 100~200 ...
    
    # 배치 내 CSV 데이터를 순회하며 그래프 구조로 변환
    for _, row in batch_df.iterrows():
        # 영화 ID 생성 (접두어 'movie-' 추가)
        movie_id = f"movie-{row['id']}"
        
        # 영화 노드 생성 (이미 존재하는지 확인하여 중복 방지)
        if movie_id not in node_dict:
            # 영화 노드 속성 설정 (추가 속성 포함)
            movie_properties = {
                "id": movie_id,  # 영화 고유 ID
                "title": row['title'],  # 영화 제목
                "released": row['released'],  # 개봉일
                "rating": float(row['rating']) if pd.notna(row['rating']) else None  #  평점 (결측값 처리)
            }
            
            # 추가 속성 처리: overview, runtime, tagline (결측값 처리)
            if pd.notna(row.get('overview')):
                movie_properties["overview"] = row['overview']
            
            if pd.notna(row.get('runtime')):
                # runtime이 숫자인 경우 정수로 변환
                try:
                    movie_properties["runtime"] = int(row['runtime'])
                except (ValueError, TypeError):
                    movie_properties["runtime"] = row['runtime']
            
            if pd.notna(row.get('tagline')):
                movie_properties["tagline"] = row['tagline']
            
            # 영화 노드 객체 생성
            movie_node = Node(
                id=movie_id,
                type="Movie",  # 노드 유형 지정
                properties=movie_properties
            )
            
            # 생성된 영화 노드를 딕셔너리에 저장
            node_dict[movie_id] = movie_node
        
        # 감독 정보 처리 (결측값이 아닌 경우에만)
        if pd.notna(row.get('director')):
            # 여러 감독이 있을 경우 '|'로 구분되어 있으므로 분리하여 처리
            for director in row['director'].split('|'):
                director = director.strip()  # 앞뒤 공백 제거
                director_id = f"person-{director}"  # 감독 ID 생성
                
                # 감독 노드가 아직 생성되지 않았다면 새로 생성
                if director_id not in node_dict:
                    director_node = Node(
                        id=director_id,
                        type="Person",  # 인물 유형으로 지정
                        properties={"name": director}  # 감독 이름 속성 설정
                    )
                    # 생성된 감독 노드를 딕셔너리에 저장
                    node_dict[director_id] = director_node
                
                # 감독과 영화 간의 'DIRECTED' 관계 생성
                relationships.append(
                    Relationship(
                        source=node_dict[director_id],  # 관계의 시작점 (감독)
                        target=node_dict[movie_id],     # 관계의 끝점 (영화)
                        type="DIRECTED",  # 관계 유형
                        properties={}  # 추가 속성 (없음)
                    )
                )
        
        # 배우 정보 처리 (결측값이 아닌 경우에만)
        if pd.notna(row.get('actors')):
            # 여러 배우가 있을 경우 '|'로 구분되어 있으므로 분리하여 처리
            for actor in row['actors'].split('|'):
                actor = actor.strip()  # 앞뒤 공백 제거
                actor_id = f"person-{actor}"  # 배우 ID 생성
                
                # 배우 노드가 아직 생성되지 않았다면 새로 생성
                if actor_id not in node_dict:
                    actor_node = Node(
                        id=actor_id,
                        type="Person",  # 인물 유형으로 지정
                        properties={"name": actor}  # 배우 이름 속성 설정
                    )
                    # 생성된 배우 노드를 딕셔너리에 저장
                    node_dict[actor_id] = actor_node
                
                # 배우와 영화 간의 'ACTED_IN' 관계 생성
                relationships.append(
                    Relationship(
                        source=node_dict[actor_id],  # 관계의 시작점 (배우)
                        target=node_dict[movie_id],  # 관계의 끝점 (영화)
                        type="ACTED_IN",  # 관계 유형
                        properties={}  # 추가 속성 (없음)
                    )
                )
        
        # 장르 정보 처리 (결측값이 아닌 경우에만)
        if pd.notna(row.get('genres')):
            # 여러 장르가 있을 경우 '|'로 구분되어 있으므로 분리하여 처리
            for genre in row['genres'].split('|'):
                genre = genre.strip()  # 앞뒤 공백 제거
                genre_id = f"genre-{genre}"  # 장르 ID 생성
                
                # 장르 노드가 아직 생성되지 않았다면 새로 생성
                if genre_id not in node_dict:
                    genre_node = Node(
                        id=genre_id,
                        type="Genre",  # 장르 유형으로 지정
                        properties={"name": genre}  # 장르 이름 속성 설정
                    )
                    # 생성된 장르 노드를 딕셔너리에 저장
                    node_dict[genre_id] = genre_node
                
                # 영화와 장르 간의 'IN_GENRE' 관계 생성
                relationships.append(
                    Relationship(
                        source=node_dict[movie_id],  # 관계의 시작점 (영화)
                        target=node_dict[genre_id],  # 관계의 끝점 (장르)
                        type="IN_GENRE",  # 관계 유형
                        properties={}  # 추가 속성 (없음)
                    )
                )
    
    print(f"배치 처리 완료: {batch_start+1}~{batch_end}/{total_rows} 레코드")

# 결과 출력
print(f"총 노드 수: {len(node_dict)}")
print(f"총 관계 수: {len(relationships)}")

배치 처리 완료: 1~100/4803 레코드
배치 처리 완료: 101~200/4803 레코드
배치 처리 완료: 201~300/4803 레코드
배치 처리 완료: 301~400/4803 레코드
배치 처리 완료: 401~500/4803 레코드
배치 처리 완료: 501~600/4803 레코드
배치 처리 완료: 601~700/4803 레코드
배치 처리 완료: 701~800/4803 레코드
배치 처리 완료: 801~900/4803 레코드
배치 처리 완료: 901~1000/4803 레코드
배치 처리 완료: 1001~1100/4803 레코드
배치 처리 완료: 1101~1200/4803 레코드
배치 처리 완료: 1201~1300/4803 레코드
배치 처리 완료: 1301~1400/4803 레코드
배치 처리 완료: 1401~1500/4803 레코드
배치 처리 완료: 1501~1600/4803 레코드
배치 처리 완료: 1601~1700/4803 레코드
배치 처리 완료: 1701~1800/4803 레코드
배치 처리 완료: 1801~1900/4803 레코드
배치 처리 완료: 1901~2000/4803 레코드
배치 처리 완료: 2001~2100/4803 레코드
배치 처리 완료: 2101~2200/4803 레코드
배치 처리 완료: 2201~2300/4803 레코드
배치 처리 완료: 2301~2400/4803 레코드
배치 처리 완료: 2401~2500/4803 레코드
배치 처리 완료: 2501~2600/4803 레코드
배치 처리 완료: 2601~2700/4803 레코드
배치 처리 완료: 2701~2800/4803 레코드
배치 처리 완료: 2801~2900/4803 레코드
배치 처리 완료: 2901~3000/4803 레코드
배치 처리 완료: 3001~3100/4803 레코드
배치 처리 완료: 3101~3200/4803 레코드
배치 처리 완료: 3201~3300/4803 레코드
배치 처리 완료: 3301~3400/4803 레코드
배치 처리 완료: 3401~3500/4803 레코드
배치 처리 

- **GraphDocument 생성 및 저장**:

    - **GraphDocument**: 그래프 전체 문서를 표현
        - `nodes`: 모든 노드의 리스트
        - `relationships`: 모든 관계의 리스트

    - 모든 노드와 관계를 수집한 후, `GraphDocument` 객체를 생성하고 이를 그래프 데이터베이스에 저장
    - GraphDocument는 **Neo4j 데이터베이스에 한 번에 저장할 수 있도록 노드와 관계를 모두 모아놓은 "패키지"

In [17]:
# 노드 딕셔너리(node_dict)에서 모든 노드 객체를 리스트 형태로 추출
# node_dict는 ID를 키로, 노드 객체를 값으로 가지고 있는 딕셔너리
nodes = list(node_dict.values())

# GraphDocument 객체 생성
# - nodes: 영화, 배우, 감독, 장르 등 모든 노드를 포함하는 리스트
# - relationships: 앞서 생성한 모든 관계(ACTED_IN, DIRECTED, IN_GENRE 등)를 포함하는 리스트
graph_doc = GraphDocument(
    nodes=nodes,
    relationships=relationships
)

# 생성된 GraphDocument를 Neo4j 데이터베이스에 저장
# add_graph_documents 메서드는 GraphDocument 객체 리스트를 받아 데이터베이스에 일괄 저장
# 이 과정에서 모든 노드와 관계가 데이터베이스에 생성됨
graph.add_graph_documents([graph_doc])
print("그래프 데이터베이스에 저장 완료")

그래프 데이터베이스에 저장 완료
